#### Note:

Make sure that you have these two scripts running at the same time while running this

ark regsitry start

python sim_node.py

In [2]:
import time
import numpy as np
from arktypes import joint_group_command_t, task_space_command_t, joint_state_t
from arktypes.utils import pack
from ark.client.comm_infrastructure.instance_node import InstanceNode

ROBOT_NAME = "arkbot"
SIM = False

class HarkControllerNode(InstanceNode):
    def __init__(self):
        super().__init__("arkbot_controller")
        suffix = "/sim" if SIM else ""
        base = f"{ROBOT_NAME}"

        # pubs
        self.joint_group_command = self.create_publisher(f"{base}/joint_group_command{suffix}", joint_group_command_t)
        self.task_space_command  = self.create_publisher(f"{base}/cartesian_command{suffix}", task_space_command_t)

        # subs
        self.state = self.create_listener(f"{base}/joint_states{suffix}", joint_state_t)

node = HarkControllerNode()

[WARNING] [13:24:24.502620] - No global configuration provided. Using default system configuration.
[INFO] [13:24:24.505008] - Service: Successfully registered '__DEFAULT_SERVICE/GetInfo/arkbot_controller_c1d7f3ed-a49f-47a2-bc83-a1e6004909d9' with registry.
[INFO] [13:24:24.507008] - Service: Successfully registered '__DEFAULT_SERVICE/SuspendNode/arkbot_controller_c1d7f3ed-a49f-47a2-bc83-a1e6004909d9' with registry.
[INFO] [13:24:24.508681] - Service: Successfully registered '__DEFAULT_SERVICE/RestartNode/arkbot_controller_c1d7f3ed-a49f-47a2-bc83-a1e6004909d9' with registry.
[ERROR] [13:24:24.509716] - Couldn't load config for other 'arkbot_controller'
[OK] [13:24:24.510595] - setup publisher arkbot/joint_group_command[joint_group_command_t]
[OK] [13:24:24.511187] - setup publisher arkbot/cartesian_command[task_space_command_t]
[OK] [13:24:24.511870] - subscribed to arkbot/joint_states[joint_state_t]


None


Get info service called
Get info service called
Get info service called


---

# Joint Group Control

In [ ]:
all_cmd = [
    1.30,   # Revolute 1 (continuous)
     0.00,   # Revolute 2 (rad)
     0.30,   # Revolute 3 (rad)
    -1.00,   # Revolute 4 (rad)
     0.10,   # Revolute 5 (continuous)
     0.80,   # Revolute 6 (rad)
     0.020   # Slider 7 position in meters (range 0.0 .. 0.0425)
]
node.joint_group_command.publish(pack.joint_group_command(all_cmd, "all"))

In [10]:
# GRIPPER (velocity mode): single value for Slider 7 velocity (m/s)
gripper_vel_open  = [ +0.010 ]   # open slowly
gripper_vel_close = [ -0.010 ]   # close slowly
node.joint_group_command.publish(pack.joint_group_command(gripper_vel_open, "gripper"))

In [7]:
# ARM (position mode): 6 values, Revolute 1..6
arm_cmd = [-0.30, 0.50, 0.30, -1.00, 0.10, 0.80]
node.joint_group_command.publish(pack.joint_group_command(arm_cmd, "arm"))


---

# Task Space Control

In [7]:
xyz_command = np.array([0.3, 0.4, 0.8])
quaternion_command = np.array([1, 0.0, 0.0, 0.0]) # xyz-w
gripper = 1.0 # 0.0 close, 1.0 open

controller.task_space_command.publish(
    pack.task_space_command("all", xyz_command, quaternion_command, gripper))

---


# Joint States

In [26]:
unpack.joint_state(controller.state.get())

({'seq': 0, 'stamp': {'sec': 0, 'nsec': 0}, 'frame_id': ''},
 ['panda_joint1',
  'panda_joint2',
  'panda_joint3',
  'panda_joint4',
  'panda_joint5',
  'panda_joint6',
  'panda_joint7',
  'panda_finger_joint1',
  'panda_finger_joint2'],
 array([ 3.32046132e-01, -2.17307877e-01,  5.52577220e-01, -2.11408249e+00,
         1.20972593e-01,  1.92591533e+00,  8.31678377e-01, -1.00911550e-21,
         3.80750550e-08]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0.]))